In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
classifier = GradientBoostingClassifier(n_estimators = 100,learning_rate = 1, max_depth = 1)

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
#----------------------------------------------------------------------------------------------------資料前處理
train_x = train[['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']]
Std = StandardScaler()
train_x[['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']] = Std.fit_transform(train_x[['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']]) #標準化
train_x[['HasCrCard','IsActiveMember']] = train_x[['HasCrCard','IsActiveMember']].astype(str)
train_x_dummies = pd.get_dummies(train_x) 
train_y = train['Exited']
#------------------------------------------------------------------------------------------------------------------------------
test_x = test[['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']]
test_x[['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']] = Std.fit_transform(test_x[['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']]) #標準化
test_x[['HasCrCard','IsActiveMember']] = test_x[['HasCrCard','IsActiveMember']].astype(str)
test_x_dummies = pd.get_dummies(test_x)
dummy = test_x_dummies.columns.tolist()

<ipython-input-2-9f643ff10bcd>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_x[['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']] = Std.fit_transform(train_x[['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']]) #標準化
c:\python37\lib\site-packages\pandas\core\indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
c:\python37\lib\site-packages\pandas\core\frame.py:3191: Settin

In [3]:
parameters = {'learning_rate': [0.01, 0.05, 0.1], 'max_depth': [2,3,4, 5, 6]}

clf = GridSearchCV(classifier, param_grid=parameters, scoring='roc_auc')
clf.fit(train_x_dummies, train_y)

pred_y =clf.predict(test_x_dummies)
#classifier.fit(train_x_dummies, train_y )
#pred_y = classifier.predict(test_x_dummies)
#pred_y = (pred_y > 0.5)
final = {"RowNumber": test["RowNumber"],'Exited':pred_y}
final_df = pd.DataFrame(final)
final_df.to_csv('final.csv')